# Equivariance Tutorial

In this tutorial, we will discuss the concept of _equivariance_. As a model problem, we consider classifying FashionMNIST, though the concepts can be easily extended to other image processing such as segmentation and denoising (and also tasks that don't involve images...). FashionMNIST is a dataset of thumbnails of items of clothing from Zalando, accompanied with a label. 

In [ ]:
import torch
from torch.nn.functional import affine_grid, grid_sample, relu, softmax, pad, conv2d
from torchvision.io import read_image
from torchvision.transforms import Normalize
from models import CNN, PDEGCNN
from lietorch.nn.m2 import LiftM2Cakewavelets
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['figure.constrained_layout.use'] = True

norm = Normalize(0.5, 0.5)
shoe = norm(read_image("images/shoe.png")[None, ...] / 255.)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.set_axis_off()
ax.set_title("Shoe")
ax.imshow(shoe.squeeze());

In [ ]:
labels = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

def logits_to_label(logits):
    return labels[logits.argmax()]

cnn_model = CNN()
cnn_model.load_state_dict(torch.load("CNN.pth", weights_only=True))
cnn_model.eval()

def cnn(image):
    with torch.no_grad():
        logits = cnn_model(image)[0]
    return logits_to_label(logits), softmax(logits, dim=-1)

pdegcnn_model = PDEGCNN()
pdegcnn_model.load_state_dict(torch.load("PDEGCNN.pth", weights_only=True))
pdegcnn_model.eval()

def pdegcnn(image):
    with torch.no_grad():
        logits = pdegcnn_model(image)[0]
    return logits_to_label(logits), softmax(logits, dim=-1)

def plot_classification(image, model, model_name, ax):
    probs = model(image)[1]
    colours = len(labels) * ["tab:blue"]
    colours[probs.argmax()] = "tab:red"
    ax.bar(labels, probs, color=colours)
    ax.set_title(model_name)
    ax.set_xlabel("Class")
    ax.set_ylim(0, 1)
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha='right')

In [ ]:
def transform(image, x, A):
    """Apply action (x, A) to image with replication padding."""
    B, C, H, W = image.shape
    x = torch.tensor([-1., 1.]) * x
    affine_matrix = torch.hstack((torch.linalg.inv(A.T), x[None, ...].T))
    grid = affine_grid(affine_matrix[None, ...], (B, C, H, W), align_corners=False)
    return grid_sample(image, grid, padding_mode="border", align_corners=False)

## Theory

### Lie Groups

> **Definition (Lie Group)** $G$ is a _Lie group_ if it is 
> 1. a _smooth manifold_ - so smooth and looks locally like $\mathbb{R}^n$ - and
> 2. a _group_ - we have a smooth, well-behaved product $\cdot: G \times G \to G$.

The most important Lie groups (imo) encode continuous symmetries on other spaces, with the group product simply given by composition.

> **Example (Translation Group)**
> The $n$-dimensional _translation group_ $\mathbb{R}^n$ acts on Euclidean space $\mathbb{R}^n$ by translation, namely
> $$ (\mathbf{x}, \mathbf{y}) \mapsto \mathbf{x} + \mathbf{y}, $$
> and has group product
> $$ (\mathbf{x}, \mathbf{y}) \mapsto \mathbf{x} + \mathbf{y}. $$
> Consequently, $\mathbb{R}^n$ also acts on the functions on Euclidean space by
> $$ (\mathbf{x}, f) \mapsto (\mathbf{y} \mapsto f(\mathbf{y} - \mathbf{x})). $$
Of course, this is an incredibly boring example. 
Slightly less trivial is the following:
> **Example (Special Orthogonal Group)**
> The _special orthogonal group_ $\operatorname{SO}(n)$ acts on Euclidean space $\mathbb{R}^n$ by rotation, namely
> $$ (R, \mathbf{y}) \mapsto R\mathbf{y}, $$
> and has group product
> $$ (R, S) \mapsto RS, $$
> Consequently, $\operatorname{SO}(n)$ also acts on the functions on Euclidean space by
> $$ (R, f) \mapsto (\mathbf{x} \mapsto f(R^{-1} \mathbf{x})). $$
Here, we represent the elements of $\operatorname{SO}(n)$ as $n \times n$ orthogonal matrices with determinant $1$. 
For example, in two dimensions the counter-clockwise rotation by angle $\theta$ is given by
$$ R = \begin{pmatrix} \cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta) \end{pmatrix}. $$

In this example the group and the space that is acted on can no longer be identified.

We get our favourite group by combining the two previous ones:
> **Example (Special Euclidean Group)**
> The _special Euclidean group_ $\operatorname{SE}(n) := \mathbb{R}^n \rtimes \operatorname{SO}(n)$ acts on Euclidean space $\mathbb{R}^n$ by roto-translation, namely
> $$ ((\mathbf{x}, R), \mathbf{y}) \mapsto \mathbf{x} + R\mathbf{y}, $$
> and has group product
> $$ ((\mathbf{x}, R), (\mathbf{y}, S)) \mapsto (\mathbf{x} + R\mathbf{y}, RS). $$
> Consequently, $\operatorname{SE}(n)$ also acts on the functions on Euclidean space by
> $$ (R, f) \mapsto (\mathbf{y} \mapsto f(R^{-1}(\mathbf{y} - \mathbf{x}))). $$

> _Remark_ The actions on functions are examples of so-called _group representations_, a term often encountered in the equivariance literature. For the sake of simplicity here we simply refer to them as actions.

Finally, we have the most simple group action: doing nothing:
> **Definition (Trivial Action)** Let $G$ be a Lie group and $X$ a set. Then we $G$ acts _trivially_ on $X$ if $g x = x$ for all $g \in G$, $x \in X$.

Many problems have inherent symmetries. For example, if we want to classify the object in an image, rotating the object shouldn't change the classification: 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].set_title("Shoe")
ax[1].set_title("Still a shoe")

x = torch.tensor([0., 0.])
R = torch.tensor([[1., 0.], [0., 1.]])
ax[0].imshow(transform(shoe, x, R).squeeze())

# Your roto-translation here ⬇️
x = torch.tensor([0., 0.])
theta = torch.tensor([0.8712346])
R = torch.tensor([
    [torch.cos(theta), -torch.sin(theta)],
    [torch.sin(theta), torch.cos(theta)]
])
ax[1].imshow(transform(shoe, x, R).squeeze());

Lie groups give us a mathematically formal way of thinking about these symmetries. In particular, we can now define _equivariance_, which in essence is a symmetry preservation property.

### Equivariance

> **Definition (Equivariance)** Let $G$ be a Lie group acting on $U$ and $V$. 
> $\Phi: U \to V$ is called _equivariant_ if it commutes with the group actions, i.e.
> $$ \Phi \circ g = g \circ \Phi, $$
> for all $g \in G$.

If the action on $V$ is trivial, then we say $\Phi$ is _invariant_. 

Let's work this out for our classification problem.
- We can see images as functions on $\mathbb{R}^2$, on which the Lie group $\operatorname{SE}(2)$ acts by roto-translation: $(\mathbf{x}, R) f(\vec{y}) = f(R^{-1} (\mathbf{y} - \mathbf{x}))$.
- We have a classifier $\Phi$ which maps an image $f: \mathbb{R}^2 \to \mathbb{R}$ to a probability distribution $p \in \mathbb{P}_c := \{(p_1, \ldots, p_c) \mid \sum_{i = 1}^c p_i = 1, p_i \geq 0\}$ over labels $\{1, \ldots, c\}$, where $c$ is the number of classes.
- $\operatorname{SE}(2)$ acts trivially on the range $\mathbb{P}_c$, so we have $(\mathbf{x}, R) p = p$ for all $p \in \mathbb{P}_c$.

Then the classifier is invariant if $\Phi(f) \circ g = g \circ \Phi(f) = \Phi(f)$ for all $g \in \operatorname{SE}(2)$ and images $f: \mathbb{R}^2 \to \mathbb{R}$.

In this problem invariance is clearly a desirable property. But how would we go about constructing an invariant classifier? 

We could train a normal convolutional neural network, and hope that it learns to be invariant. This is highly unlikely, unless we perform _data augmentation_, and even then there are no guarantees.

Alternatively, we could construct a neural network architecture that is inherently invariant. For this, we can make use of the following result:
> **Lemma (Composition of Equivariant Maps)** Let $G$ be a Lie group acting on $U$, $V$, and $W$. Suppose $\Phi: U \to V$ and $\Psi: V \to W$ are equivariant. Then, their composition $\Psi \circ \Phi: U \to W$ is also equivariant. 

_proof_: Simply note that $\Psi \circ \Phi \circ g = \Psi \circ g \circ \Phi = g \circ \Psi \circ \Phi$.

Hence, we can make an equivariant neural network architecture by composing equivariant layers. A typical layer in a neural network consists of the composition of something linear (e.g. matrix multiplication, convolution, linear combinations) with something nonlinear (e.g. activation function, normalisation). Common nonlinearities such as the ReLU activation function and batch normalisation act point-wise; it is not hard to see that such point-wise operations are equivariant.

In [ ]:
x = torch.tensor([0., 0.])
theta = torch.tensor([-0.862364]) # Random number
R = torch.tensor([
    [torch.cos(theta), -torch.sin(theta)],
    [torch.sin(theta), torch.cos(theta)]
])

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].set_title("Transform before ReLU")
ax[1].set_title("ReLU before Transform")
ax[0].imshow(relu(transform(shoe, x, R)).squeeze())
ax[1].imshow(transform(relu(shoe), x, R).squeeze());

We're working with images in this tutorial; convolutions are a natural choice for the linear part. In a convolutional layer an image is convolved/cross-correlated with a trainable filter (Fig. 5.3 from [1]):
![Schematic depiction of a convolution.](convolution.png)
Convolutional layers are translation equivariant: if you shift the input image, the output is shifted accordingly (up to boundary effects...):

In [ ]:
kernel = torch.randn(1, 1, 3, 3)

k = 4 # number of pixels to shift
padding = k + 1 # deal with boundary issues
x = torch.tensor([k * (2 / (shoe.shape[-1] + 2 * padding)), 0.])
theta = torch.tensor([0.])
R = torch.tensor([
    [torch.cos(theta), -torch.sin(theta)],
    [torch.sin(theta), torch.cos(theta)]
])

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].set_title("Shift before Convolution")
ax[1].set_title("Convolution before Shift")
ax[0].imshow(conv2d(transform(pad(shoe, 4*[padding], mode="replicate"), x, R), kernel, torch.tensor([0.])).squeeze())
ax[1].imshow(transform(conv2d(pad(shoe, 4*[padding], mode="replicate"), kernel, torch.tensor([0.])), x, R).squeeze())

However, convolutional layers typically won't be rotation equivariant.

In [ ]:
x = torch.tensor([0., 0.])
theta = torch.tensor([torch.pi/2])
R = torch.tensor([
    [torch.cos(theta), -torch.sin(theta)],
    [torch.sin(theta), torch.cos(theta)]
])

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].set_title("Rotate before Convolution")
ax[1].set_title("Convolution before Rotate")
ax[0].imshow(conv2d(transform(shoe, x, R), kernel, torch.tensor([0.])).squeeze())
ax[1].imshow(transform(conv2d(shoe, kernel, torch.tensor([0.])), x, R).squeeze())

Indeed, convolutions are rotation equivariant if and only if the kernel is isotropic. We can achieve this by averaging a normal (anisotropic) kernel over all possible rotations:

In [ ]:
isotropic_kernel = (
    kernel + 
    torch.rot90(kernel, dims=(-2, -1)) + 
    torch.rot90(kernel, k=-1, dims=(-2, -1)) +
    torch.rot90(torch.rot90(kernel, dims=(-2, -1)), dims=(-2, -1))
) / 4

x = torch.tensor([0., 0.])
theta = torch.tensor([torch.pi/2])
R = torch.tensor([
    [torch.cos(theta), -torch.sin(theta)],
    [torch.sin(theta), torch.cos(theta)]
])

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].set_title("Rotate before Convolution")
ax[1].set_title("Convolution before Rotate")
ax[0].imshow(conv2d(transform(shoe, x, R), isotropic_kernel, torch.tensor([0.])).squeeze())
ax[1].imshow(transform(conv2d(shoe, isotropic_kernel, torch.tensor([0.])), x, R).squeeze());

One limitation of this approach is that we are greatly limiting the number of models that can be expressed with the same number of parameters. For example, if we rotate the initial kernel, then that will lead to the same isotropic kernel, even though the kernel parameters are different. This issue can be addressed by _lifting_.

### Lifting

Up to now, we have seen images as functions that map the plane $\mathbb{R}^2$ to a scalar in $\mathbb{R}$. Roughly speaking, the gray value (or maybe the change thereof) at a given point is a measure of the presence of structure at that location. However, when we look at an image, we can say more. 

In [ ]:
cross = torch.zeros(9, 9, 3)
cross[4, 1:8] = 1.
cross[1:8, 4] = 1.
cross[4, 2] = torch.tensor([1., 0., 0.])
cross[2, 4] = torch.tensor([0., 0., 1.])

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.set_axis_off()
ax.imshow(cross);

In this simple image of a cross, we can see that the red point is horizontally oriented, while the blue dot is vertically oriented. In other words, at the position of the red dot and the horizontal orientation, there is structure present, but not at e.g. the position of the red dot and the vertical orientation. Hence, we can ascribe to every position and orientation a gray value; we call this an _orientation score_
> **Definition (Orientation Score Transform)** Let $\psi : \mathbb{R}^2 \to \mathbb{R}$ be a wavelet. Then the _orientation score transform_ $\mathcal{W}_\psi$ maps an image $f : \mathbb{R}^2 \to \mathbb{R}$ to an _orientation score_ $\mathcal{W}_\psi f : \mathbb{R}^2 \times S^1 \cong \operatorname{SE}(2) \to \mathbb{R}$ via 
> $$ \mathcal{W}_\psi f(\mathbf{x}, \theta) = \int_{\mathbb{R}^2} ((\mathbf{x}, R_\theta) \psi)(\mathbf{y}) f(\mathbf{y}) \mathrm{d} \mathbf{y} = \int_{\mathbb{R}^2} \psi(R_\theta^{-1} (\mathbf{y} - \mathbf{x})) f(\mathbf{y}) \mathrm{d} \mathbf{y}. $$
Classically, we choose the filter wavelet $\psi$ to pick up horizontally oriented features; then the gray scale values in the "image" $\mathcal{W}_\psi f(\cdot, \theta): \mathbb{R}^2 \to \mathbb{R}$ are a measure for the presence of a feature at some position with orientation $\theta$:

In [ ]:
Or = 16
cross = torch.zeros(1, 1, 64, 64)
cross[..., 31:34, 8:56] = 1.
cross[..., 8:56, 31:34] = 1.
cakewavelet_lift = LiftM2Cakewavelets(cross.shape[-3], Or, inflection_point=1.)
lifted_cross = cakewavelet_lift(cross)

k = 0
dtheta = 2 * torch.pi / Or
fig, ax = plt.subplots(1, 4, figsize=(20, 5))
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()
ax[3].set_axis_off()
ax[0].set_title("$f$")
ax[1].set_title(fr"$\mathcal{{W}}_\psi f(\cdot , {k * dtheta / torch.pi:.2f} \pi)$")
ax[2].set_title(fr"$\mathcal{{W}}_\psi f(\cdot , {(k+Or//4) * dtheta / torch.pi:.2f} \pi)$")
ax[3].set_title(r"$\sum_{\theta \in S^1} \mathcal{W}_\psi f(\cdot , \theta)$")
ax[0].imshow(cross.squeeze())
ax[1].imshow(lifted_cross[0, 0, k])
ax[2].imshow(lifted_cross[0, 0, k+(Or//4)])
ax[3].imshow(lifted_cross.sum(-3).squeeze());

The orientation score transform is equivariant for any wavelet $\psi$:
> **Lemma (Orientation Score Transform is Equivariant)**
> Let $\psi, f: \mathbb{R}^2 \to \mathbb{R}$, and let $g \in \operatorname{SE}(2)$. Then,
> $$ g (\mathcal{W}_\psi f) = \mathcal{W}_\psi (g f). $$
_proof:_ We can simply rewrite:
$$\begin{split}
(\mathbf{x}, R) (\mathcal{W}_\psi f)(\mathbf{y}, S) & := \mathcal{W}_\psi f ((\mathbf{x}, R)^{-1} (\mathbf{y}, S)) = \int_{\mathbb{R}^2} (\mathbf{x}, R)^{-1} (\mathbf{y}, S) \psi(\mathbf{z}) f(\mathbf{z}) \mathrm{d} \mathbf{z} = \int_{\mathbb{R}^2} \psi((\mathbf{y}, S)^{-1} (\mathbf{x}, R) \mathbf{z}) f(\mathbf{z}) \mathrm{d} \mathbf{z} \\
& = \int_{\mathbb{R}^2} \psi((\mathbf{y}, S)^{-1} \mathbf{v}) f((\mathbf{x}, R)^{-1} \mathbf{v}) \mathrm{d} (\mathbf{x}, R)^{-1} \mathbf{v} \overset{(1)}{=} \int_{\mathbb{R}^2} \psi((\mathbf{y}, S)^{-1} \mathbf{v}) f((\mathbf{x}, R)^{-1} \mathbf{v}) \mathrm{d} \mathbf{v} \\
& = \mathcal{W}_\psi ((\mathbf{x}, R) f) (\mathbf{y}, S),
\end{split}$$
where we used the left-invariance of the standard Lebesgue measure in $(1)$.

We now have a way of lifting images from $\mathbb{R}^2$ to orientation scores on the group $\operatorname{SE}(2)$. The next step is to find equivariant operators on such orientation scores. It turns out that convolutions can be generalised to group convolutions. 
> **Definition (Group Convolution)** Let $G$ be a Lie group, and let $f, k : G \to \mathbb{R}$ be functions thereon. Then we define the _group convolution_ of $f$ and $k$ as
> $$ (k * f)(g) := \int_G K(h^{-1} g) f(h) \mathrm{d} h. $$

To get a feel for group convolutions, consider the following examples.
> **Example (Convolution on the Translation Group)**
> Let $f, k : \mathbb{R}^2 \to \mathbb{R}$. Then, 
> $$ (k * f)(\mathbf{x}) := \int_{\mathbb{R}^2} k(\mathbf{x} - \mathbf{y}) f(\mathbf{y}) \mathrm{d} \mathbf{y}. $$

> **Example (Convolution on the Special Orthogonal Group)**
> Let $f, k : \operatorname{SO}(2) \cong S^1 \to \mathbb{R}$. Then, 
> $$ (k * f)(\theta) := \int_{\operatorname{SO}(2)} k(S^{-1} R_\theta) f(S) \mathrm{d} S = \int_{S^1} k(\theta - \phi) f(\phi) \mathrm{d} \phi. $$

> **Example (Convolution on the Special Euclidean Group)**
> Let $f, k : \operatorname{SE}(2) \cong \mathbb{R}^2 \times S^1 \to \mathbb{R}$. Then, 
> $$ (k * f)(\mathbf{x}, \theta) := \int_{\operatorname{SE}(2)} k((\mathbf{y}, S)^{-1} (\mathbf{x}, R_\theta)) f(\mathbf{y}, S) \mathrm{d} \mathbf{y} \mathrm{d} S = \int_{\mathbb{R}^2 \times S^1} k(R_{\phi}^{-1} (\mathbf{x} - \mathbf{y}), \theta - \phi) f(\mathbf{y}, \phi) \mathrm{d} \mathbf{y} \mathrm{d} \phi. $$

In the same way that normal convolutions - which are defined on functions on Euclidean space/translation group - are translation equivariant, group convolutions are equivariant to the corresponding group.
> **Lemma (Group Convolutions are Equivariant)**
> Let $G$ be a Lie group, let $f, k : G \to \mathbb{R}$ be functions thereon, and let $r \in G$. Then, 
> $$ r (k * f) = k * (r f). $$
_proof:_ We can simply rewrite:
$$\begin{split}
r (k * f)(g) & := (k * f)(r^{-1} g) := \int_G K(h^{-1} r^{-1} g) f(h) \mathrm{d} h = \int_G k((r h)^{-1} g) f(h) \mathrm{d} h \\
& = \int_G k(q^{-1} g) f(r^{-1} q) \mathrm{d} r^{-1}q \overset{(1)}{=} \int_G k(q^{-1} g) f(r^{-1} q) \mathrm{d} q \\
& =  (k * (r f))(g),
\end{split}$$
where we used the left-invariance of the Haar measure in $(1)$.

In practice, we usually want our output to be an image again, so we need a way to equivariantly map orientation scores to images. We call this projection. We typically use max projection:
> **Definition (Max Projection)** Let $f : \operatorname{SE}(2) \to \mathbb{R}$. Then we define the _max projection_ of $f$
> $$ \operatorname{Proj} f(\mathbf{x}, R) := \max_{R \in \operatorname{SO}(2)} f(\mathbf{x}, R). $$
It is not hard to see that max projection is indeed equivariant:
$$\begin{split}
(\mathbf{x}, R) (\operatorname{Proj} f)(\mathbf{y}) & := \operatorname{Proj} f((\mathbf{x}, R)^{-1} \mathbf{y}) = \operatorname{Proj} f(R^{-1}(\mathbf{y} - \mathbf{x})) \\
& = \max_{S \in \operatorname{SO}(2)} f(R^{-1}(\mathbf{y} - \mathbf{x}), S) = \max_{S \in \operatorname{SO}(2)} f(R^{-1}(\mathbf{y} - \mathbf{x}), R^{-1} S) = \max_{S \in \operatorname{SO}(2)} (\mathbf{x}, R) f(\mathbf{y}, S) \\
& = (\operatorname{Proj} (\mathbf{x}, R) f)(\mathbf{y}).
\end{split}$$

Hence, one way to make a roto-translation equivariant neural network architecture is as follows:
1. Start with a lifting layer that maps images on $\mathbb{R}^2$ to orientation scores on the group $\operatorname{SE}(2)$.
2. Subsequently apply $\operatorname{SE}(2)$ group convolutions and point-wise nonlinearities in alternating fashion.
3. Project the orientation scores back to images using max projection.

Neural networks with this architecture, which mirrors the classical multi-orientation image processing pipeline [2], are called Group equivariant Convolutional Neural Networks (G-CNNs).
![Multi-orientation processing pipeline: first lift with the orientation score transform, then perform equivariant processing on the orientation scores, and finally project back down to an image.](multi-orientation_processing.png)

For our classification task, the output should not be an image but a "vector" in $\mathbb{P}_c := \{(p_1, \ldots, p_c) \mid \sum_{i = 1}^c p_i = 1, p_i \geq 0\}$, on which the group $\operatorname{SE}(2)$ acts trivially. We can achieve this by adding additional layers to our network. The output images need to be invariantly converted to numbers, e.g. by taking the maximum over the image, which can then be combined in whatever manner we like. 

Similar approaches can be taken for other affine groups, such as the translation-scaling group and the similarity group. One limitation of lifting is that it increases memory use: if we use $K$ discrete orientations, then the orientation score will use $K$ times as much memory as the image. This problem gets worse for higher dimensional groups, such as the similarity group $\operatorname{SIM}(2) := \mathbb{R}^2 \rtimes (\operatorname{SO}(2) \times \mathbb{R}_+)$, the group of translations, rotations and scalings: if we use $K$ discrete orientations and $M$ discrete scales, memory use will be $K \times M$ times as large. In other words, memory use scales exponentially in the dimension of the group. Consequently, it is important to choose only the most important symmetries to integrate into the architecture.

## Application

This repository contains two trained models: a CNN and a roto-translation invariant PDE-G-CNN. The architectures can be found in the [`models.py`](models.py) module. The models have trained using the [`train.py`](train.py) script.

The CNN is based on the classic LeNet-5 architecture. It consists of two convolutional layers with max-pooling and ReLU activation function, which are (approximately) translation equivariant, followed by three fully connected layers. Notably, the first fully connected layer is _not_ translation invariant, and we also do not have guaranteed translation invariance on the whole model.

For our invariant network, we use a PDE-based Group equivariant CNN (PDE-G-CNN) architecture, which were developed in our group by Smets et al. [3]. They are closely related to the Group equivariant CNNs (G-CNNs) described in the previous section. They have a couple of special features:

1. Recall that in a G-CNN we alternate the linear convolution layers with point-wise nonlinearities. This is done because alternating linear and nonlinear layers turns out to be necessary to get expressive models. In PDE-G-CNNs, the nonlinear layers no longer act point-wise, but instead are nonlinear convolutions.
2. The convolutional layers (approximately) solve PDEs from classical image processing: the convolution kernels are defined in terms of the parameters of these PDEs.

It has been shown in various works that PDE-G-CNNs can achieve performance competitive with e.g. CNNs, but with a large reduction in the number of model parameters. Additionally, they tend to be more data efficient, so they can be applied in situations where data is not abundant.

![Comparison of a traditional CNN layer and a PDE-G-CNN layer.](layer_comparison.webp)

In [ ]:
x = torch.tensor([0., 0.])
theta = torch.tensor([torch.pi/2])
R = torch.tensor([
    [torch.cos(theta), -torch.sin(theta)],
    [torch.sin(theta), torch.cos(theta)]
])

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
plot_classification(shoe, pdegcnn, "PDE-G-CNN", ax[0, 0])
plot_classification(transform(shoe, x, R), pdegcnn, "PDE-G-CNN", ax[0, 1])
plot_classification(shoe, cnn, "CNN", ax[1, 0])
plot_classification(transform(shoe, x, R), cnn, "CNN", ax[1, 1])

# References

<a id="Smets2024GeometricProcessing"></a>[1] B.M.N. Smets. Geometric Partial Differential Equations in Deep Learning and Image Processing (2024). <https://research.tue.nl/en/publications/geometric-partial-differential-equations-in-deep-learning-and-ima>

```bib
@phdthesis{smets2024geometric,
  title={Geometric Partial Differential Equations in Deep Learning and Image Processing},
  author={Smets, Bart M.N.},
  year={2024},
  isbn={978-90-386-6133-9},
}
```

<a id="Sherry2025DiffusionSpace"></a>[2] F.M. Sherry, K. Schaefer, R. Duits. Diffusion-Shock PDEs for Deep Learning on Position-Orientation Space. arXiv preprint (2025). <https://doi.org/10.48550/arXiv.2509.06405>

```bib
@article{sherry2025diffusion,
  title={Diffusion-Shock PDEs for Deep Learning on Position-Orientation Space},
  author={Sherry, F.M. and Schaefer, K. and Duits, R.},
  journal={arXiv preprint},
  year={2025},
  doi={10.48550/arXiv.2509.06405},
}
```

<a id="Smets2022PDENetworks"></a>[3] B.M.N. Smets, J. Portegies, E.J. Bekkers, R. Duits. PDE-Based Group Equivariant Convolutional Neural Networks. J Math Imaging Vis (2022). <https://doi.org/10.1007/s10851-022-01114-x>

```bib
@article{smets2022pde,
  title={PDE-based Group Equivariant Convolutional Neural Networks},
  author={Smets, Bart M.N. and Portegies, Jim and Bekkers, Erik J. and Duits, Remco},
  journal={Journal of Mathematical Imaging and Vision},
  publisher={Springer},
  year={2022},
  doi={10.1007/s10851-022-01114-x},
}
```